In [100]:
import re
import string

operators = {
    '+': (1, lambda a, b: a + b),
    '*': (2, lambda a, b: a * b) # 1 for part 1, 2 for part 2
}

operators_by_precedence = [
        {o: f for o, (p, f) in operators.items() if p == i}
    for i in sorted(set(v[0] for v in operators.values()))
]

def tokenize(expr, i=0):
    tokens = []
    while i < len(expr):
        if expr[i].isspace():
            i += 1
        elif expr[i] == '(':
            i, value = tokenize(expr, i+1)
            tokens.append(value)
            assert expr[i] == ')', 'invalid syntax'
            i += 1
        elif m := re.search('^[0-9]+', expr[i:]):
            tokens.append(int(m.group()))
            i += len(m.group())
        elif expr[i] in operators:
            tokens.append(expr[i])
            i += 1
        else:
            break
    return i, tokens

def tokens2ast(tokens):
    tokens = list(tokens)
    
    for i, token in enumerate(tokens):
        if isinstance(token, list):
            tokens[i] = tokens2ast(token)
    
    for operators in operators_by_precedence:
        i = 0
        while i < len(tokens):
            if tokens[i] in operators:
                node = (tokens[i], tokens[i-1], tokens[i+1])
                tokens = tokens[:i-1] + [node] + tokens[i+2:]
            else:
                i += 1
    
    assert len(tokens) == 1, 'invalid syntax'
    return tokens[0]

def parse(expr):
    i, tokens = tokenize(expr)
    assert i == len(expr), 'not everything was parsed'
    return tokens2ast(tokens)

def eval_ast(ast):
    if isinstance(ast, int):
        return ast
    op, a, b = ast
    a = eval_ast(a)
    b = eval_ast(b)
    
    assert op in operators, 'invalid operator'
    
    return operators[op][1](a, b)
    
    
def parse_eval(expr):
    ast = parse(expr)
    return eval_ast(ast)

In [101]:
parse_eval('((2 + 4 * 9) * (6 + 9 * 8 + 6) + 6) + 2 + 4 * 2')

23340

In [102]:
with open('day18.txt') as d:
    s = 0
    for l in d:
        s += parse_eval(l)
    print(s)

360029542265462
